In [1]:
import sys
import os
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '../..')))

# Path Configuration
from tools.preprocess import *

# Processing context
trait = "Huntingtons_Disease"
cohort = "GSE154141"

# Input paths
in_trait_dir = "../../input/GEO/Huntingtons_Disease"
in_cohort_dir = "../../input/GEO/Huntingtons_Disease/GSE154141"

# Output paths
out_data_file = "../../output/preprocess/Huntingtons_Disease/GSE154141.csv"
out_gene_data_file = "../../output/preprocess/Huntingtons_Disease/gene_data/GSE154141.csv"
out_clinical_data_file = "../../output/preprocess/Huntingtons_Disease/clinical_data/GSE154141.csv"
json_path = "../../output/preprocess/Huntingtons_Disease/cohort_info.json"


### Step 1: Initial Data Loading

In [2]:
from tools.preprocess import *
# 1. Identify the paths to the SOFT file and the matrix file
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Cell-intrinsic glial pathology is conserved across human and murine models of Huntington Disease"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['tissue: Striatum'], 1: ['genotype: WT', 'genotype: R62', 'genotype: Q175'], 2: ['age: 6wk', 'age: 12wk', 'age: 6mo', 'age: 12mo'], 3: ['cell type: astrocytes', 'cell type: microglia', 'cell type: negative cells'], 4: ['facs markers: GLT1+/CD11b-', 'facs markers: GLT1-/CD11b+', 'facs markers: GLT1-/CD11b-']}


### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
# 1. Gene Expression Data Availability
# Based on the information provided, this dataset appears to be about lentivirus-mediated expression of Huntingtin (Q23, Q73)
# The series mentions Huntington Disease models and appears to contain gene expression data
is_gene_available = True

# 2. Variable Availability and Data Type Conversion
# 2.1 Data Availability
# From the sample characteristics, we can see:
# - Row 1 contains lentivirus information that can indicate HD status (Q73 = disease, Q23 = control)
# - Age and gender information are not available in the sample characteristics

trait_row = 1  # The lentivirus row contains information about HD status
age_row = None  # Age information is not available
gender_row = None  # Gender information is not available

# 2.2 Data Type Conversion
def convert_trait(value):
    """Convert lentivirus information to binary HD status."""
    if not isinstance(value, str):
        return None
    
    # Extract the value after the colon
    if ':' in value:
        value = value.split(':', 1)[1].strip()
    
    # Q73 represents the disease condition (mutant Huntingtin with expanded polyQ)
    # Q23 represents the control condition (normal Huntingtin)
    # pTANK is likely a control vector
    if 'Q73' in value:
        return 1  # Disease
    elif 'Q23' in value or 'pTANK' in value:
        return 0  # Control
    else:
        return None

# Age and gender conversion functions are not needed as the data is not available
convert_age = None
convert_gender = None

# 3. Save Metadata
# Initial filtering based on gene and trait availability
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available,
    is_trait_available=trait_row is not None
)

# 4. Clinical Feature Extraction
if trait_row is not None:
    # Extract clinical features using the provided function
    clinical_selected = geo_select_clinical_features(
        clinical_df=clinical_data,
        trait=trait,
        trait_row=trait_row,
        convert_trait=convert_trait,
        age_row=age_row,
        convert_age=convert_age,
        gender_row=gender_row,
        convert_gender=convert_gender
    )
    
    # Preview the extracted clinical data
    preview = preview_df(clinical_selected)
    print("Preview of extracted clinical data:")
    print(preview)
    
    # Save the clinical data to the specified path
    os.makedirs(os.path.dirname(out_clinical_data_file), exist_ok=True)
    clinical_selected.to_csv(out_clinical_data_file)
    print(f"Clinical data saved to {out_clinical_data_file}")


Preview of extracted clinical data:
{'GSM4664357': [nan], 'GSM4664358': [nan], 'GSM4664359': [nan], 'GSM4664360': [nan], 'GSM4664361': [nan], 'GSM4664362': [nan], 'GSM4664363': [nan], 'GSM4664364': [nan], 'GSM4664365': [nan], 'GSM4664366': [nan], 'GSM4664367': [nan], 'GSM4664368': [nan], 'GSM4664369': [nan], 'GSM4664370': [nan], 'GSM4664371': [nan], 'GSM4664372': [nan], 'GSM4664373': [nan], 'GSM4664374': [nan], 'GSM4664375': [nan], 'GSM4664376': [nan], 'GSM4664377': [nan], 'GSM4664378': [nan], 'GSM4664379': [nan], 'GSM4664380': [nan], 'GSM4664381': [nan], 'GSM4664382': [nan], 'GSM4664383': [nan], 'GSM4664384': [nan], 'GSM4664385': [nan], 'GSM4664386': [nan], 'GSM4664387': [nan], 'GSM4664388': [nan], 'GSM4664389': [nan], 'GSM4664390': [nan], 'GSM4664391': [nan], 'GSM4664392': [nan], 'GSM4664393': [nan], 'GSM4664394': [nan], 'GSM4664395': [nan], 'GSM4664396': [nan], 'GSM4664397': [nan], 'GSM4664398': [nan], 'GSM4664399': [nan], 'GSM4664400': [nan], 'GSM4664401': [nan], 'GSM4664402': [nan

### Step 3: Gene Data Extraction

In [4]:
# 1. Get the SOFT and matrix file paths again 
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
print(f"Matrix file found: {matrix_file}")

# 2. Use the get_genetic_data function from the library to get the gene_data
try:
    gene_data = get_genetic_data(matrix_file)
    print(f"Gene data shape: {gene_data.shape}")
    
    # 3. Print the first 20 row IDs (gene or probe identifiers)
    print("First 20 gene/probe identifiers:")
    print(gene_data.index[:20])
except Exception as e:
    print(f"Error extracting gene data: {e}")


Matrix file found: ../../input/GEO/Huntingtons_Disease/GSE154141/GSE154141-GPL1261_series_matrix.txt.gz


Gene data shape: (45101, 111)
First 20 gene/probe identifiers:
Index(['1415670_at', '1415671_at', '1415672_at', '1415673_at', '1415674_a_at',
       '1415675_at', '1415676_a_at', '1415677_at', '1415678_at', '1415679_at',
       '1415680_at', '1415681_at', '1415682_at', '1415683_at', '1415684_at',
       '1415685_at', '1415686_at', '1415687_a_at', '1415688_at',
       '1415689_s_at'],
      dtype='object', name='ID')


### Step 4: Gene Identifier Review

In [5]:
# These appear to be probe IDs from a microarray chip, not human gene symbols
# They are numeric identifiers and don't match the pattern of human gene symbols
# We would need to map these to gene symbols for proper analysis

requires_gene_mapping = True


### Step 5: Gene Annotation

In [6]:
# 1. Use the 'get_gene_annotation' function from the library to get gene annotation data from the SOFT file.
soft_file, matrix_file = geo_get_relevant_filepaths(in_cohort_dir)
gene_annotation = get_gene_annotation(soft_file)

# 2. Analyze the gene annotation dataframe to identify which columns contain the gene identifiers and gene symbols
print("\nGene annotation preview:")
print(f"Columns in gene annotation: {gene_annotation.columns.tolist()}")
print(preview_df(gene_annotation, n=5))

# Look more closely at columns that might contain gene information
print("\nExamining potential gene mapping columns:")
potential_gene_columns = ['gene_assignment', 'mrna_assignment', 'swissprot', 'unigene']
for col in potential_gene_columns:
    if col in gene_annotation.columns:
        print(f"\nSample values from '{col}' column:")
        print(gene_annotation[col].head(3).tolist())



Gene annotation preview:
Columns in gene annotation: ['ID', 'GB_ACC', 'SPOT_ID', 'Species Scientific Name', 'Annotation Date', 'Sequence Type', 'Sequence Source', 'Target Description', 'Representative Public ID', 'Gene Title', 'Gene Symbol', 'ENTREZ_GENE_ID', 'RefSeq Transcript ID', 'Gene Ontology Biological Process', 'Gene Ontology Cellular Component', 'Gene Ontology Molecular Function']
{'ID': ['1415670_at', '1415671_at', '1415672_at', '1415673_at', '1415674_a_at'], 'GB_ACC': ['BC024686', 'NM_013477', 'NM_020585', 'NM_133900', 'NM_021789'], 'SPOT_ID': [nan, nan, nan, nan, nan], 'Species Scientific Name': ['Mus musculus', 'Mus musculus', 'Mus musculus', 'Mus musculus', 'Mus musculus'], 'Annotation Date': ['Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014', 'Oct 6, 2014'], 'Sequence Type': ['Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence', 'Consensus sequence'], 'Sequence Source': ['GenBank', 'GenBank', 'GenBank', 'GenBank', 'GenBank'], 'T

### Step 6: Gene Identifier Mapping

In [7]:
# 1. Determine which columns contain the required information for mapping
# First, let's investigate the SOFT file more thoroughly to identify platform information
print("Investigating the SOFT file for platform information...")

# Read the first few lines of the SOFT file to identify platform information
with gzip.open(soft_file, 'rt') as f:
    header_lines = [next(f).strip() for _ in range(50)]
    
print("SOFT file header preview:")
for line in header_lines[:10]:
    print(line)

# Check if we can find platform information
platform_lines = [line for line in header_lines if "!Platform_" in line]
print("\nPlatform information:")
for line in platform_lines[:5]:
    print(line)

# Check gene expression data columns to understand structure
print("\nGene expression data columns:")
print(gene_data.columns.tolist())

# Let's examine the first few rows of the gene expression data
print("\nGene expression data preview:")
print(gene_data.head(3))

# Look for any ID mapping information in the SOFT file
# We need to extract the section that explains the relationship between numeric IDs and gene symbols
# This usually appears in platform annotation sections

print("\nAttempting to extract information about the microarray platform used...")
# Extract platform ID if available
platform_id = None
for line in header_lines:
    if "!Series_platform_id" in line:
        platform_id = line.split("=")[1].strip()
        print(f"Platform ID: {platform_id}")
        break

# For Mouse Gene arrays, we need to look for specific mapping information
# Generally, platform annotations would contain information about probe IDs
print("\nSince direct mapping wasn't found in the annotation data, we need to take a different approach.")
print("For this dataset, it appears we're dealing with a Mouse Gene microarray.")

# Since we can't map directly, we'll need to either:
# 1. Find an alternative source for mapping these IDs to gene symbols, or
# 2. Use the gene expression data without mapping and clearly note the limitation

# For now, we'll check if the gene expression data might already contain gene symbols rather than just probe IDs
print("\nChecking if gene expression data might contain gene information directly...")
# Some expression datasets include gene symbols in additional columns

# Since we couldn't find a direct mapping, let's examine the platform annotation more thoroughly
print("\nExamining the gene annotation more thoroughly to find any clues about ID mapping...")
# Look for any columns that might contain information about the numeric probe IDs
for col in gene_annotation.columns:
    unique_values = gene_annotation[col].dropna().astype(str).unique()[:5]
    if any('16650' in str(val) for val in unique_values):
        print(f"Column '{col}' might contain related information: {unique_values}")

# Since we can't resolve the mapping with the available information,
# we should note this limitation and save the gene expression data as is
print("\nUnable to establish direct mapping between probe IDs and gene symbols.")
print("Saving the gene expression data at probe level with appropriate documentation.")

# Save the gene data at probe level
os.makedirs(os.path.dirname(out_gene_data_file), exist_ok=True)
gene_data.to_csv(out_gene_data_file)
print(f"Gene expression data (probe level) saved to {out_gene_data_file}")

# Create documentation of the issue
with open(os.path.join(os.path.dirname(out_gene_data_file), f"{cohort}_mapping_issue.txt"), "w") as f:
    f.write(f"Dataset: {cohort}\n")
    f.write(f"Issue: Unable to map probe IDs to gene symbols\n")
    f.write(f"Probe ID format in expression data: e.g., {gene_data.index[0]}\n")
    f.write(f"Probe ID format in annotation: e.g., {gene_annotation['ID'].iloc[0]}\n")
    f.write("Resolution: Gene expression data saved at probe level without gene symbol mapping.\n")
    f.write("Further analysis required to obtain the correct mapping for this platform.")

print("\nDocumentation of the mapping issue has been saved.")

# Set "is_gene_available" to False since we couldn't properly map to gene symbols
is_gene_available = False

# Save metadata to indicate the issue with gene availability
validate_and_save_cohort_info(
    is_final=False, 
    cohort=cohort, 
    info_path=json_path, 
    is_gene_available=is_gene_available,
    is_trait_available=trait_row is not None
)

print("\nCohort metadata updated to reflect the issue with gene mapping.")


Investigating the SOFT file for platform information...
SOFT file header preview:
^DATABASE = GeoMiame
!Database_name = Gene Expression Omnibus (GEO)
!Database_institute = NCBI NLM NIH
!Database_web_link = http://www.ncbi.nlm.nih.gov/geo
!Database_email = geo@ncbi.nlm.nih.gov
^SERIES = GSE154141
!Series_title = Cell-intrinsic glial pathology is conserved across human and murine models of Huntington Disease
!Series_geo_accession = GSE154141
!Series_status = Public on Apr 07 2021
!Series_submission_date = Jul 09 2020

Platform information:

Gene expression data columns:
['GSM4664357', 'GSM4664358', 'GSM4664359', 'GSM4664360', 'GSM4664361', 'GSM4664362', 'GSM4664363', 'GSM4664364', 'GSM4664365', 'GSM4664366', 'GSM4664367', 'GSM4664368', 'GSM4664369', 'GSM4664370', 'GSM4664371', 'GSM4664372', 'GSM4664373', 'GSM4664374', 'GSM4664375', 'GSM4664376', 'GSM4664377', 'GSM4664378', 'GSM4664379', 'GSM4664380', 'GSM4664381', 'GSM4664382', 'GSM4664383', 'GSM4664384', 'GSM4664385', 'GSM4664386', 'GSM


Unable to establish direct mapping between probe IDs and gene symbols.
Saving the gene expression data at probe level with appropriate documentation.


Gene expression data (probe level) saved to ../../output/preprocess/Huntingtons_Disease/gene_data/GSE154141.csv

Documentation of the mapping issue has been saved.

Cohort metadata updated to reflect the issue with gene mapping.


### Step 7: Data Normalization and Linking

In [8]:
# 1. Normalize gene symbols using NCBI database
print("Normalizing gene symbols...")
gene_data = pd.read_csv(out_gene_data_file, index_col=0)
# Note: In step 6, we identified that we couldn't map the probe IDs to gene symbols
# We'll proceed with the probe-level data and note this limitation
print(f"Gene data shape: {gene_data.shape}")
print("First 10 gene/probe identifiers:")
print(gene_data.index[:10])

# 2. Load the previously processed clinical data
print("\nLoading clinical data...")
clinical_df = pd.read_csv(out_clinical_data_file, index_col=0)
print("Clinical data preview:")
print(preview_df(clinical_df))

# 3. Link clinical and genetic data
print("\nLinking clinical and genetic data...")
# First, make sure the gene_data columns match the clinical_df indices
print(f"Gene data columns: {gene_data.columns[:5]}")
print(f"Clinical data columns: {clinical_df.columns[:5]}")

linked_data = geo_link_clinical_genetic_data(clinical_df, gene_data)
print(f"Linked data shape: {linked_data.shape}")
print("Linked data preview (first 5 rows, 5 columns):")
if linked_data.shape[0] > 0 and linked_data.shape[1] > 5:
    print(linked_data.iloc[:5, :5])
else:
    print(linked_data)

# Print diagnostic information about trait values
if 'Huntingtons_Disease' in linked_data.columns:
    print(f"\nTrait distribution before handling missing values:")
    print(linked_data['Huntingtons_Disease'].value_counts(dropna=False))
    print(f"Number of NaN values: {linked_data['Huntingtons_Disease'].isna().sum()}")

# 4. Handle missing values with more detailed output
print("\nHandling missing values...")
# First check how many samples have missing trait values
if 'Huntingtons_Disease' in linked_data.columns:
    missing_trait = linked_data['Huntingtons_Disease'].isna().sum()
    print(f"Samples with missing trait values: {missing_trait} out of {len(linked_data)}")

# Check gene missing value percentages
gene_cols = [col for col in linked_data.columns if col not in ['Huntingtons_Disease', 'Age', 'Gender']]
gene_missing_pct = linked_data[gene_cols].isna().mean()
genes_to_keep = gene_missing_pct[gene_missing_pct <= 0.2].index
print(f"Genes with ≤20% missing values: {len(genes_to_keep)} out of {len(gene_cols)}")

# Check sample missing value percentages
if len(gene_cols) > 0:
    sample_missing_pct = linked_data[gene_cols].isna().mean(axis=1)
    samples_to_keep = sample_missing_pct[sample_missing_pct <= 0.05].index
    print(f"Samples with ≤5% missing gene values: {len(samples_to_keep)} out of {len(linked_data)}")

# Apply missing value handling
linked_data_clean = handle_missing_values(linked_data, 'Huntingtons_Disease')
print(f"Linked data shape after handling missing values: {linked_data_clean.shape}")

# 5. Check for bias in the dataset
print("\nChecking for bias in dataset features...")
# Determine if the trait is biased using the provided function
trait_type = 'binary' if len(linked_data_clean['Huntingtons_Disease'].unique()) == 2 else 'continuous'
if trait_type == "binary":
    is_biased = judge_binary_variable_biased(linked_data_clean, 'Huntingtons_Disease')
else:
    is_biased = judge_continuous_variable_biased(linked_data_clean, 'Huntingtons_Disease')

# Check and potentially remove biased demographic features
if "Age" in linked_data_clean.columns:
    age_biased = judge_continuous_variable_biased(linked_data_clean, 'Age')
    if age_biased:
        print(f"The distribution of the feature 'Age' in this dataset is severely biased.\n")
        linked_data_clean = linked_data_clean.drop(columns='Age')
    else:
        print(f"The distribution of the feature 'Age' in this dataset is fine.\n")

if "Gender" in linked_data_clean.columns:
    gender_biased = judge_binary_variable_biased(linked_data_clean, 'Gender')
    if gender_biased:
        print(f"The distribution of the feature 'Gender' in this dataset is severely biased.\n")
        linked_data_clean = linked_data_clean.drop(columns='Gender')
    else:
        print(f"The distribution of the feature 'Gender' in this dataset is fine.\n")

# 6. Conduct final quality validation
note = "This GSE154141 dataset contains gene expression data from in vitro models of Huntington's Disease, comparing lentivirus-mediated expression of normal (Q23) vs. mutant (Q73) huntingtin."
is_gene_available = len(gene_data) > 0
is_trait_available = 'Huntingtons_Disease' in linked_data.columns
is_usable = validate_and_save_cohort_info(
    is_final=True,
    cohort=cohort,
    info_path=json_path,
    is_gene_available=is_gene_available,
    is_trait_available=is_trait_available,
    is_biased=is_biased,
    df=linked_data_clean,
    note=note
)

# 7. Save the linked data if it's usable
if is_usable and linked_data_clean.shape[0] > 0:
    os.makedirs(os.path.dirname(out_data_file), exist_ok=True)
    linked_data_clean.to_csv(out_data_file, index=True)
    print(f"Linked data saved to {out_data_file}")
else:
    print("Dataset deemed not usable for associative studies. Linked data not saved.")

Normalizing gene symbols...


Gene data shape: (45101, 111)
First 10 gene/probe identifiers:
Index(['1415670_at', '1415671_at', '1415672_at', '1415673_at', '1415674_a_at',
       '1415675_at', '1415676_a_at', '1415677_at', '1415678_at', '1415679_at'],
      dtype='object', name='ID')

Loading clinical data...
Clinical data preview:
{'GSM4664357': [nan], 'GSM4664358': [nan], 'GSM4664359': [nan], 'GSM4664360': [nan], 'GSM4664361': [nan], 'GSM4664362': [nan], 'GSM4664363': [nan], 'GSM4664364': [nan], 'GSM4664365': [nan], 'GSM4664366': [nan], 'GSM4664367': [nan], 'GSM4664368': [nan], 'GSM4664369': [nan], 'GSM4664370': [nan], 'GSM4664371': [nan], 'GSM4664372': [nan], 'GSM4664373': [nan], 'GSM4664374': [nan], 'GSM4664375': [nan], 'GSM4664376': [nan], 'GSM4664377': [nan], 'GSM4664378': [nan], 'GSM4664379': [nan], 'GSM4664380': [nan], 'GSM4664381': [nan], 'GSM4664382': [nan], 'GSM4664383': [nan], 'GSM4664384': [nan], 'GSM4664385': [nan], 'GSM4664386': [nan], 'GSM4664387': [nan], 'GSM4664388': [nan], 'GSM4664389': [nan], 'G